## Bitcoin Allocation Strategies
Comparing different allocation strategies historically

. Lump-Sum

. Dollar Cost Averaging (DCA)

. Combination (optimized)

In [142]:
# Import Libraries
import pandas as pd
import numpy as np
import urllib
import requests
import riskfolio as rp
from datetime import datetime, timedelta

# Chart libraries + settings
%matplotlib inline
import matplotlib.pyplot as plt
# plt.style.use('seaborn-whitegrid')
pd.options.mode.chained_assignment = None  # default='warn' - disable some pandas warnings


ModuleNotFoundError: No module named 'riskfolio'

In [130]:
# Load Bitcoin Prices into a dataframe
# Ticker is customizable
ticker = "BTC"
# Cryptocompare URL and fiels
base_url = 'https://min-api.cryptocompare.com/data/histoday'
ticker_field = 'fsym'
field_dict = {'tsym': 'USD','allData': 'true'}
# Convert the field dict into a url encoded string
url_args = "&" + urllib.parse.urlencode(field_dict)
ticker = ticker.upper()
globalURL = (base_url + "?" + ticker_field + "=" + ticker + url_args)


In [131]:
# Request the data
resp = requests.get(url=globalURL)
data = resp.json()
data["Response"]

KeyboardInterrupt: 

In [8]:
# Parse the JSON into a Pandas DataFrame
try:
    df = pd.DataFrame.from_dict(data['Data'])
    df = df.rename(columns={'time': 'date'})
    df['date'] = pd.to_datetime(df['date'], unit='s')
    df.set_index('date', inplace=True)
    df_save = df[['close', 'open', 'high', 'low']]
except Exception as e:
    self.errors.append(e)
    df_save = None

In [9]:
# Include percentage change 
df = df_save
df['change'] = df['close'].pct_change()

### Support Utilities to be used later

In [141]:
# Increment n number of months of certain date
def monthdelta(date, delta):
    m, y = (date.month+delta) % 12, date.year + ((date.month)+delta-1) // 12
    if not m: m = 12
    d = min(date.day, [31,
        29 if y%4==0 and not y%400==0 else 28,31,30,31,30,31,31,30,31,30,31][m-1])
    new_date = (date.replace(day=d,month=m, year=y))
    return new_date

def add_periods(date, periods, frequency):
    if frequency.upper() == 'D' or 'DAY' in frequency.upper():
        return (date + timedelta(days=periods))
    if frequency.upper() == 'W' or 'WEEK' in frequency.upper():
        return (date + timedelta(days=periods * 7))
    if frequency.upper() == 'M' or 'MONTH' in frequency.upper():
        return(monthdelta(date, periods))
    if frequency.upper() == 'Y' or 'YEAR' in frequency.upper():
        return(monthdelta(date, periods * 12))
    
def annualization_factor(df):
    """
    Receives a df and returns the number of periods to apply
    to annualize the returns. For BTC this should be close to
    365 as it trades daily. For stocks should be close to 252.
    Args:
        df (_type_): _description_
    """
    start_date = df.index[0]
    end_date = df.index[-1]
    number_of_days = (end_date - start_date).days
    fraction_of_year = number_of_days / 365
    data_points = len(df)
    annualization_factor = data_points / fraction_of_year
    return int(round(annualization_factor, 0))


### Main Allocation Class
See example on creating an allocation instance at the cell following the class definition

In [138]:
class AllocationManager:
    def __init__(self):
        self.frequency = 'D'  # 'D', 'W', 'M', 'Y'
        self.allocation_periods = 30  # Assume allocation happens during 30 periods
        self.upfront_percent = 0  # [0 - 1]: amount to be allocated upfront
        self.capital = 100000  # 10,000 dollars to allocate
        self.df = df  # Bitcoin Prices Dataframe
        self.start_date = self.df.index.min()  # Date where allocation starts (default = first date)
        self.end_date = self.df.index.max()  # End date for analysis (default = today), but this can be used to test specific timeframes (ex: ending last year)
        # Create empty allocation & position columns
        self.df['allocation'] = 0
        self.df['BTC_tx'] = 0
        self.risk_free_rate = 0.05
         
    def allocate_capital(self):
        # TRIM THE DF between start and end dates
        # TO DO ---------------------------
        
        # Updates the dataframe to allocate the capital
        available_capital = self.capital 
        current_date = self.start_date
        periods_left = self.allocation_periods
        
        # Set upfront amount if any & per period amounts
        if self.upfront_percent > 0:
            upfront = self.upfront_percent * self.capital  # how much upfront in $
            per_period = (self.capital - upfront) / (self.allocation_periods - 1)
        else:
            per_period = self.capital / self.allocation_periods 
            upfront = per_period
        
        # Start looping until allocation is complete
        while periods_left > 0:
            # Allocate Capital
            if current_date == self.start_date:
                self.df.at[current_date, 'allocation'] = upfront
            else:
                self.df.at[current_date, 'allocation'] = per_period
            
            # Allocate BTC
            self.df.at[current_date, 'BTC_tx'] = (
                self.df.at[current_date, 'allocation'] / 
                self.df.at[current_date, 'close'] 
                )
            current_date = add_periods(current_date, 1, self.frequency)
            periods_left -= 1

        # Sum all BTC Txs and calculate portfolio values
        self.df['BTC_position'] = self.df['BTC_tx'].cumsum()
        self.df['portfolio_position'] = (self.df['BTC_position'] * self.df['close'])
        self.df = self.df.fillna(0)
        # TO DO ---------------------
        self.df['cum_capital'] = self.df['allocation'].cumsum()
        self.df['normalized_port_position'] = (self.df['portfolio_position'] / 
                                               self.df['cum_capital']) * 100
        self.df['cum_return'] = self.df['normalized_port_position'] / 100

    
    def stats(self):
        self.allocate_capital()
        stats = {}
        df = self.df
        stats['capital allocated'] = df.allocation.sum()
        stats['BTC allocated'] = df.BTC_tx.sum()
        stats['max portfolio value'] = df.portfolio_position.max()
        stats['final portfolio value'] = df.portfolio_position[-1]
        # Calculate Return on Invested Capital
        stats['ROIC'] = (
            (stats['final portfolio value'] / 
             stats['capital allocated']) - 1
            )
        # Calculate Multiple of Invested Capital
        stats['MOIC'] = (
            (stats['final portfolio value'] / 
             stats['capital allocated'])
            )
        
        stats['number_points'] = df['change'].count()
        stats['volatility_daily'] = df['change'].std()
        stats['annualization_factor'] = annualization_factor(df)
        stats['volatility_annual'] = df['change'].std() * annualization_factor(df)**.5
        stats['return_annual'] = ((df['cum_return'][-1]) ** (annualization_factor(df) / df['change'].count())) -1
    
        stats['sharpe_ratio'] = (stats['return_annual'] -
                             self.risk_free_rate) / stats['volatility_annual']
        Y = df['normalized_port_position'].pct_change().dropna()
        stats['hist_VaR_95'] = rp.RiskFunctions.VaR_Hist(Y, alpha=0.05)
        stats['max_DD_abs'] = rp.RiskFunctions.MDD_Rel(Y)
        stats['avg_DD_abs'] = rp.RiskFunctions.ADD_Rel(Y)
        stats['DaR_DD_abs'] = rp.RiskFunctions.DaR_Rel(Y, alpha=0.05)        
        return (stats)

In [139]:
# Create a sample Instance of the AllocationManager and test results
btc_alloc = AllocationManager()
btc_alloc.capital = 100
btc_alloc.allocation_periods = 4
btc_alloc.frequency = 'D'
btc_alloc.upfront_percent = 0.9
btc_alloc.allocate_capital()

In [140]:
btc_alloc.df

,close,open,high,low,change,allocation,BTC_position,BTC_tx,portfolio_position,cum_capital,normalized_port_position
date,,,,,,,,,,,
2010-07-17,0.04951,0.04951,0.04951,0.04951,0.000000,90.000000,1817.814583,1817.814583,9.000000e+01,90.000000,1.000000e+02
2010-07-18,0.08584,0.04951,0.08585,0.05941,0.733791,3.333333,1856.646518,38.831935,1.593745e+02,93.333333,1.707584e+02
2010-07-19,0.08080,0.08584,0.09307,0.07723,-0.058714,3.333333,1897.900644,41.254125,1.533504e+02,96.666667,1.586383e+02
2010-07-20,0.07474,0.08080,0.08181,0.07426,-0.075000,3.333333,1942.499698,44.599055,1.451824e+02,100.000000,1.451824e+02
2010-07-21,0.07921,0.07474,0.07921,0.06634,0.059807,0.000000,1942.499698,0.000000,1.538654e+02,100.000000,1.538654e+02
...,...,...,...,...,...,...,...,...,...,...,...
2023-01-11,17938.00000,17442.44000,17986.45000,17324.71000,0.028411,0.000000,1942.499698,0.000000,3.484456e+07,100.000000,3.484456e+07
2023-01-12,18849.00000,17938.00000,19092.31000,17906.40000,0.050786,0.000000,1942.499698,0.000000,3.661418e+07,100.000000,3.661418e+07
2023-01-13,19932.05000,18849.00000,19992.23000,18717.45000,0.057459,0.000000,1942.499698,0.000000,3.871800e+07,100.000000,3.871800e+07


### Run some quick checks

In [136]:
# Show only the allocation periods
al_df = btc_alloc.df.where(btc_alloc.df.allocation > 0).dropna()
display(al_df)
# Check it adds to allocation amount
print("Total allocation:")
print(round(al_df.allocation.sum(), 8))
if (round(al_df.allocation.sum(), 8) == round(btc_alloc.capital, 8)):
    print ("Checks [OK]")
else:
    print ("[ERROR] - something went wrong")

,close,open,high,low,change,allocation,BTC_position,BTC_tx,portfolio_position
date,,,,,,,,,
2010-07-17,0.04951,0.04951,0.04951,0.04951,0.000000,90.000000,1817.814583,1817.814583,90.000000
2010-07-18,0.08584,0.04951,0.08585,0.05941,0.733791,3.333333,1856.646518,38.831935,159.374537
2010-07-19,0.08080,0.08584,0.09307,0.07723,-0.058714,3.333333,1897.900644,41.254125,153.350372
2010-07-20,0.07474,0.08080,0.08181,0.07426,-0.075000,3.333333,1942.499698,44.599055,145.182427


Total allocation:
100.0
Checks [OK]


In [137]:
# Show statistics
btc_alloc.stats()

{'capital allocated': 100.0,
 'BTC allocated': 1942.4996982087885,
 'max portfolio value': 131214184.0642632,
 'final portfolio value': 40655450.30867593,
 'ROIC': 406553.5030867593,
 'MOIC': 406554.5030867593}